# Reporte abonos cartolas BECH

In [ ]:
import os
import datetime
import tkinter as tk
from tkinter import filedialog
import pandas as pd
from openpyxl import Workbook
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import numpy as np

username = os.getlogin()            
hoy = datetime.datetime.now()
date_format = "%d.%m.%Y"
date_formatexport = "%d-%m-%Y"
fecha_actual = hoy.strftime(date_formatexport)
fecha_cartola = hoy.strftime(date_format)

df_mes = pd.DataFrame
df_ajustes = pd.DataFrame
alm = pd.DataFrame

lowmemory = False

file_path = 'C:\\Users\\' + username + '\\Documents\\Py\\IDENTIFICADOR\\AJUSTES\\MONTOSXAGENCIA.csv'

print("Cargando archivos...")

df_cierres = pd.read_csv(file_path,
                        sep=';',
                        header=0,
                        dtype=np.int64,
                        names=['monto','agencia'])

folder_path = 'C:\\Users\\' + username + '\\Documents\\Py\\ARCHIVOS PROCESO AM\\CARTOLAS HISTORICAS'

files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
df_list = []
for file in files:
    df = pd.read_csv(os.path.join(folder_path, file),
                        sep=';',
                        header=None,
                        thousands='.',
                        names=['fecha', 'sucursal', 'operacion', 'descripcion', 'cargos', 'abonos','ind'])
    df_list.append(df)
df_mes = pd.concat(df_list)
df_mes['fecha'] = df_mes['fecha'].str.replace('/', '-')
df_mes['fecha'] = pd.to_datetime(df_mes['fecha'],format='mixed',dayfirst=True)
df_mes = df_mes.sort_values(by='fecha')

df_agencias = df_cierres
df_agencias = df_agencias[df_agencias['monto'] != 0]
df_agencias[['monto','agencia']] = df_agencias[['monto','agencia']].astype(int)
df_agencias['agencia'] = df_agencias['agencia'].astype(int)
df_agencias['monto'] = df_agencias['monto'].astype(int)
df_agencias.drop_duplicates(subset='monto',keep='first', inplace=True)
df_agencias.sort_values(by='monto', ascending=True, inplace=True)


df_ident = pd.merge(df_mes, df_agencias, how='left', left_on='abonos', right_on='monto')
df_ident['agencia'] = df_ident['agencia'].astype(str).replace('\.0$','',regex=True)
df_ident['monto'] = df_ident['monto'].astype(str).replace('\.0$','',regex=True)
df_ident['fecha'] = df_ident['fecha'].dt.strftime(date_formatexport)
df_ident['abonos'] = df_ident['abonos'].astype(int)
df_ident = df_ident.drop(columns=['ind','monto'])

print("Archivos cargados")

In [2]:

df_ident.to_excel('C:\\Users\\' + username + '\\Documents\\Py\\JPNB\\GENERADOS\\Reporte mes ' + fecha_actual + '.xlsx',
            # header=True,
            index=False)

In [2]:
df_agencias.to_excel('C:\\Users\\' + username + '\\Documents\\Py\\JPNB\\GENERADOS\\AGENCIASXMONTO.xlsx',
            header=True,
            index=False)

# Pie de abonos

In [ ]:
df_pie = pd.DataFrame(columns=['tabono','count'])
count_tf = df_mes[df_mes['descripcion'].str.contains('transferencia', case=False)].shape[0]
count_def = df_mes[df_mes['descripcion'].str.contains('efectivo', case=False)].shape[0]
count_otros = df_mes[~df_mes['descripcion'].str.contains('efectivo|transferencia', case=False,regex=True)].shape[0]
df_pie.loc[len(df_pie)]= ['Transferencia',count_tf]
df_pie.loc[len(df_pie)]= ['Depósito en efectivo',count_def]
#df_pie.loc[len(df_pie)]= ['Otros',count_otros]
df_pie.set_index('tabono',inplace=True)
df_pie.plot.pie(y='count',ylabel = '',title = 'Abonos',autopct= lambda p:'{:.1f}%'.format(p,p * sum(df_pie['count'])/100), figsize=(10,5))


Conteo tipos de abonos

In [ ]:
df_muestra = df_mes

df_pietransf = pd.DataFrame(columns=['tabono','count'])

df_tf = df_muestra[df_muestra['descripcion'].str.contains('transferencia', case=False)]
df_tfob = df_tf[df_tf['descripcion'].str.startswith('Transferencia otro banco de rut')]
df_tfbech = df_tf[df_tf['descripcion'].str.startswith('Transferencia bancoestado de')]
df_tfcv = df_tf[df_tf['descripcion'].str.startswith('Transferencia caja vecina')]

count_tfob = df_tfob.shape[0]
count_tfbech = df_tfbech.shape[0]
count_tfcv = df_tfcv.shape[0]

df_pietransf.loc[len(df_pietransf)]= ['CajaVecina',count_tfcv]
df_pietransf.loc[len(df_pietransf)]= ['Banco Estado',count_tfbech]
df_pietransf.loc[len(df_pietransf)]= ['Otro banco',count_tfob]
df_pietransf.set_index('tabono',inplace=True)
df_pietransf.plot.pie(y='count',ylabel='', title='Transferencias',autopct= lambda p:'{:.1f}%'.format(p,p * sum(df_pie['count'])/100), figsize=(10,5))

df_piedef = pd.DataFrame(columns=['tabono','count'])

df_def = df_muestra[df_muestra['descripcion'].str.contains('efectivo', case=False)]
df_defef = df_def[df_def['descripcion'].str.startswith('Depositoenefectivo')]
df_defsv = df_def[df_def['descripcion'].str.startswith('Deposito en efectivo serviestado')]
df_defcv = df_def[df_def['descripcion'].str.startswith('Deposito en efectivo caja vecina')]
df_defcaj = df_def[df_def['descripcion'].str.startswith('Deposito efectivo en cajero automatico')]

count_defef = df_defef.shape[0]
count_defsv = df_defsv.shape[0]
count_defcv = df_defcv.shape[0]
count_defcaj = df_defcaj.shape[0]

df_piedef.loc[len(df_piedef)]= ['Sucursal',count_defef]
df_piedef.loc[len(df_piedef)]= ['CajaVecina',count_defcv]
df_piedef.loc[len(df_piedef)]= ['ServiEstado',count_defsv]
#df_piedef.loc[len(df_piedef)]= ['Otros',count_defcaj]
df_piedef.set_index('tabono',inplace=True)
df_piedef.plot.pie(y='count',ylabel='',title='Depósitos en efectivo',startangle=180,autopct= lambda p:'{:.1f}%'.format(p,p * sum(df_pie['count'])/100), figsize=(10,5))

In [ ]:
df_conteo = df_mes['fecha'].value_counts().reset_index(name='conteo')
df_conteo = df_conteo[df_conteo['fecha'] < '2024-12-24']
df_conteo = df_conteo[df_conteo['fecha'] > '2024-06-01']
df_conteo.set_index('fecha',inplace=True)
df_conteo.plot(title='Registros diarios',xlabel='mes',ylabel='Conteo abonos',figsize=(20,6))
#plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p:f'{x}'))
myFmt = mdates.DateFormatter('%d/%b/%Y')
plt.gca().xaxis.set_major_formatter(myFmt)
plt.xticks(np.arange('2024-11', '2025-01', dtype='datetime64[W]'),rotation=10, ha='right')
plt.grid(True)
plt.yticks(np.arange(1000,2500,step=100))
plt.gca().minorticks_on()
plt.show()
print('Promedio: ' + df_conteo['conteo'].mean().astype(str))

# Ubicar abono según N° Operacion

In [ ]:
df_muestra = df_mes[df_mes['operacion'].astype(str).str.contains('1562', case=False)].shape[0]
# sorted_df = count_df.sort_values(by='descripcion', ascending=False)
print (df_muestra)

# Cuadratura de abonos

In [ ]:
#muestraex = df_mes[df_mes['abonos'].astype(int) == 49180]
#muestrarut = df_mes[df_mes['descripcion'].str.contains() | df_mes['descripcion'].str.contains('')]
muestrarut = df_mes[df_mes['descripcion'].str.contains('1')]
muestra = pd.concat([muestraex,muestrarut])
print(muestra)


: 

In [ ]:

df_adosc = df_ident[df_ident['abonos'] == 200000]

conteo_adosc = df_adosc.groupby(['fecha']).count()

df_adosc_plot = conteo_adosc.copy().resample('M').sum()
print(df_adosc_plot)

In [ ]:
freq_adosc.plot(kind='bar',x='frequency', figsize=(10,4))
plt.title('Casos Pre-conciliados')
plt.xlabel('Periodo')
plt.xticks(rotation=30, ha='right')
plt.ylabel('Casos', fontsize = 14)
plt.yticks(np.arange(0,850,step=50))
plt.grid(axis='y')
arrcasos = df_adosc.to_numpy()
#for i, v in enumerate(y):
#    plt.text(xlocs[i] - 0.25, v + 0.01, str(v))
#myFmt = mdates.DateFormatter('%b/%Y')
#plt.gca().xaxis.set_major_formatter(myFmt)
#plt.xticks(np.arange('2023-12', '2024-12', dtype='datetime64[M]'),rotation=10, ha='right')
plt.show()